In [1]:
import logging

import config
import lightgbm as lgb
import polars as pl
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor
from lifelines import BreslowFlemingHarringtonFitter, KaplanMeierFitter
from sklearn.model_selection import train_test_split

from src.customs.fold import add_kfold
from src.customs.metrics import CatBoostMetric, Metric, ROCAUCMetric
from src.feature.tabular import AggregateEncoder, OrdinalEncoder, RawEncoder
from src.model.sklearn_like import (
    CatBoostClassifierWrapper,
    CatBoostRegressorWrapper,
    LightGBMWapper,
    XGBoostRegressorWrapper,
)
from src.trainer.tabular.simple import single_inference_fn_v2, single_train_fn

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [2]:
def load_data(valid_ratio: float | None = None) -> pl.DataFrame:
    raw_train_df = pl.read_csv(config.COMP_DATASET_DIR / "train.csv").with_columns(
        pl.lit("TRAIN").alias(config.DATASET_COL),
        pl.lit(-1).alias(config.FOLD_COL),
    )
    if valid_ratio is not None:
        _tr_df, _va_df = train_test_split(raw_train_df, test_size=0.2, random_state=config.SEED)
        _va_df = _va_df.with_columns(
            pl.lit("VALID").alias(config.DATASET_COL),
        )
        raw_train_df = pl.concat([_tr_df, _va_df], how="diagonal_relaxed")

    target_df = pl.read_csv(config.INPUT_DIR / config.EXTRA_TARGET_CSV)
    target_cols = [x for x in target_df.columns if x.startswith("target_")]
    raw_train_df = raw_train_df.join(target_df, on=config.ID_COL, how="left")

    raw_test_df = pl.read_csv(config.COMP_DATASET_DIR / "test.csv").with_columns(
        pl.lit("TEST").alias(config.DATASET_COL)
    )

    config.META_COLS = sorted(list(set(config.META_COLS + target_cols)))

    train_test_df = pl.concat([raw_train_df, raw_test_df], how="diagonal_relaxed").sort(
        config.DATASET_COL, config.ID_COL, descending=[True, False]
    )
    return train_test_df


def fe(
    train_test_df: pl.DataFrame,
    output_dataset: str = "TRAIN",
) -> pl.DataFrame:
    encoders = [
        RawEncoder(columns=config.META_COLS, prefix=""),
        RawEncoder(
            columns=(
                [
                    *config.NUMERICAL_COLS,
                ]
            ),
            prefix=f"{config.FEATURE_PREFIX}n_",
        ),
        OrdinalEncoder(
            columns=(
                [
                    *config.CATEGORICAL_COLS,
                ]
            ),
            prefix=f"{config.FEATURE_PREFIX}c_",
        ),
    ]

    for encoder in encoders:
        encoder.fit(train_test_df.filter(pl.col(config.DATASET_COL) == "TRAIN"))

    features_df = pl.concat(
        [
            encoder.transform(
                train_test_df.filter(pl.col(config.DATASET_COL) == output_dataset),
            )
            for encoder in encoders
        ],
        how="horizontal",
    )
    return features_df

In [ ]:
train_test_df = load_data(valid_ratio=config.VALID_RATIO)
features_df = fe(train_test_df)

feature_names = sorted([x for x in features_df.columns if x.startswith(config.FEATURE_PREFIX)])
cat_features = [x for x in feature_names if x.startswith(f"{config.FEATURE_PREFIX}c_")]


logger.info(f"# of features: {len(feature_names)}")
logger.info(f"# of cat_features: {len(cat_features)}")

In [ ]:
train_features_df = add_kfold(
    features_df,
    n_splits=config.N_SPLITS,
    random_state=config.SEED,
    fold_col=f"{config.FOLD_COL}_nested",
)


all_result_df = pl.DataFrame()
for i_fold in train_features_df[f"{config.FOLD_COL}_nested"].unique().to_list():
    logger.info(f"Outer Fold: {i_fold}")
    tr_df = train_features_df.filter(pl.col(f"{config.FOLD_COL}_nested") != i_fold)
    va_df = train_features_df.filter(pl.col(f"{config.FOLD_COL}_nested") == i_fold)
    print(tr_df.shape, va_df.shape)
    _, _, trained_models = single_train_fn(
        model=CatBoostRegressorWrapper(
            name=f"cat_{i_fold}",
            model=CatBoostRegressor(
                loss_function="Cox",
                learning_rate=0.1,
                n_estimators=10000,
                early_stopping_rounds=100,
                eval_metric=CatBoostMetric(),
                verbose=100,
                # subsample=0.5,
                colsample_bylevel=0.2,
                random_state=config.SEED,
                # task_type="GPU",
                reg_lambda=5,
            ),
            multi_output=False,
            feature_names=feature_names,
            cat_features=cat_features,
        ),
        features_df=add_kfold(
            tr_df,
            n_splits=config.N_SPLITS,
            random_state=config.SEED,
            fold_col=config.FOLD_COL,
        ),
        feature_cols=feature_names,
        target_col="target_cox",
        fold_col=config.FOLD_COL,
        meta_cols=config.META_COLS,
        # weight_col="sample_weight",
        out_dir=config.OUTPUT_DIR,
        train_folds=None,
        eval_fn=Metric(),
        overwrite=False,
        use_eval_metric_extra_va_df=True,
    )
    va_result_df = single_inference_fn_v2(
        models=trained_models,
        features_df=va_df,
        feature_names=feature_names,
    )

    all_result_df = pl.concat(
        [
            all_result_df,
            va_result_df.select(
                pl.col(config.ID_COL),
                pl.col("pred"),
            ),
        ],
        how="diagonal_relaxed",
    )


# INFO:src.trainer.tabular.simple:✅ Final Metric: 0.6897873116621929
# INFO:src.trainer.tabular.simple:✅ Final Metric: 0.6898640681195222


In [ ]:
all_result_df

In [ ]:
def make_new_targets(
    df: pl.DataFrame,
    base_target_names: tuple[str] = ("target_kmf", "target_bfhf"),
) -> pl.DataFrame:
    lower_bound = 0.0

    for base_target_name in base_target_names:
        new_target_name = f"{base_target_name}_event_pred_scaled"

        scaling_factor = df.select(pl.col("pred").log().min() / (lower_bound - pl.col(base_target_name).min()))[
            "pred"
        ].to_numpy()[0]
        print(scaling_factor)

        new_df = df.select(
            pl.col(config.ID_COL),
            (pl.col("pred").log() / scaling_factor + pl.col(base_target_name)).alias(new_target_name),
        )
        df = df.join(new_df, on=config.ID_COL, how="left")
    return df


result_df = (
    train_features_df.join(all_result_df, on=config.ID_COL, how="left")
    .select(
        [
            pl.col("ID"),
            pl.col("efs"),
            pl.col("efs_time"),
            pl.col("target_kmf"),
            pl.col("target_bfhf"),
            pl.col("pred").alias("target_cox_pred"),
            (((pl.col("pred")) - pl.col("pred").min()) / (pl.col("pred").max() - pl.col("pred").min())).alias("pred")
            + 0.01,
        ]
    )
    .with_columns(
        pl.col("pred").alias("target_cox_pred_normalized"),
        (pl.col("target_kmf").exp() + pl.col("target_cox_pred").exp()).log().alias("target_cox_kfm"),
    )
)
result_df = make_new_targets(result_df)

result_df.write_csv(config.INPUT_DIR / f"{config.EXP_NAME}_target.csv")